In [6]:
import pandas as pd
df = pd.read_csv("/Users/moo/Desktop/MSc Business Analytics/BEMM466 Business project/BERTopic Finalised/arxiv_ai_cleaned_ready.csv")
df.head(10)

,authors,categories,comment,doi,entry_id,journal_ref,pdf_url,primary_category,published,summary,...,updated,abstract_len_words,year,month,abstract_len_words_raw,authors_list,author_names,n_authors,summary_clean,abstract_len_words_clean
0,[arxiv.Result.Author('M. L. Ginsberg')],['cs.AI'],See http://www.jair.org/ for an online appendi...,NaN,http://arxiv.org/abs/cs/9308101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9308101v1,cs.AI,1993-08-01 00:00:00+00:00,Because of their occasional need to return to ...,...,1993-08-01 00:00:00+00:00,79,1993,8,79,"[""[arxiv.Result.Author('M. L. Ginsberg')]""]",['M. L. Ginsberg'],1,occasional need return shallow point search tr...,49
1,[arxiv.Result.Author('M. P. Wellman')],['cs.AI'],See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9308102v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9308102v1,cs.AI,1993-08-01 00:00:00+00:00,Market price systems constitute a well-underst...,...,1993-08-01 00:00:00+00:00,119,1993,8,119,"[""[arxiv.Result.Author('M. P. Wellman')]""]",['M. P. Wellman'],1,market price system constitute wellunderstood ...,74
2,"[arxiv.Result.Author('I. P. Gent'), arxiv.Resu...",['cs.AI'],See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9309101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9309101v1,cs.AI,1993-09-01 00:00:00+00:00,We describe an extensive study of search in GS...,...,1993-09-01 00:00:00+00:00,167,1993,9,167,"[""[arxiv.Result.Author('I. P. Gent'), arxiv.Re...","['I. P. Gent', 'T. Walsh']",2,describe extensive study search gsat approxima...,96
3,"[arxiv.Result.Author('F. Bergadano'), arxiv.Re...",['cs.AI'],See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9311101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9311101v1,cs.AI,1993-11-01 00:00:00+00:00,As real logic programmers normally use cut (!)...,...,1993-11-01 00:00:00+00:00,174,1993,11,174,"[""[arxiv.Result.Author('F. Bergadano'), arxiv....","['F. Bergadano', 'D. Gunetti', 'U. Trinchero']",3,real logic programmer normally use cut effecti...,94
4,"[arxiv.Result.Author('J. C. Schlimmer'), arxiv...",['cs.AI'],See http://www.jair.org/ for an online appendi...,NaN,http://arxiv.org/abs/cs/9311102v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9311102v1,cs.AI,1993-11-01 00:00:00+00:00,To support the goal of allowing users to recor...,...,1993-11-01 00:00:00+00:00,187,1993,11,187,"[""[arxiv.Result.Author('J. C. Schlimmer'), arx...","['J. C. Schlimmer', 'L. A. Hermens']",2,support goal allowing user record retrieve inf...,113
5,"[arxiv.Result.Author('M. Buchheit'), arxiv.Res...",['cs.AI'],See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9312101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9312101v1,cs.AI,1993-12-01 00:00:00+00:00,Terminological knowledge representation system...,...,1993-12-01 00:00:00+00:00,161,1993,12,161,"[""[arxiv.Result.Author('M. Buchheit'), arxiv.R...","['M. Buchheit', 'F. M. Donini', 'A. Schaerf']",3,terminological knowledge representation system...,102
6,[arxiv.Result.Author('N. Nilsson')],['cs.AI'],See http://www.jair.org/ for any accompanying ...,NaN,http://arxiv.org/abs/cs/9401101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9401101v1,cs.AI,1994-01-01 00:00:00+00:00,A formalism is presented for computing and org...,...,1994-01-01 00:00:00+00:00,144,1994,1,144,"[""[arxiv.Result.Author('N. Nilsson')]""]",['N. Nilsson'],1,formalism presented computing organizing actio...,74
7,[arxiv.Result.Author('C. X. Ling')],['cs.AI'],See http://www.jair.org/ for an online appendi...,NaN,http://arxiv.org/abs/cs/9402101v1,"Journal of Artificial Intelligence Research, V...",http://arxiv.org/pdf/cs/9402101v1,cs.AI,1994-02-01 00:00:00+00:00,Learning the past tense of English verbs

In [8]:
docs = df['summary_clean'].astype(str).tolist()

In [10]:
from bertopic import BERTopic

# Load the saved model
topic_model = BERTopic.load("bertopic_model_trial_8")

In [12]:
# Apply topic reduction (creates a new model)
reduced_model = topic_model.reduce_topics(docs, nr_topics=20)

In [36]:
# Get topic info for the reduced model
reduced_topic_info = reduced_model.get_topic_info()

total_docs = len(docs)

n_topics = (reduced_topic_info['Topic'] != -1).sum()

# Number of noise documents (if topic -1 exists)
if -1 in reduced_topic_info['Topic'].values:
    noise_count = reduced_topic_info[reduced_topic_info['Topic'] == -1]['Count'].values[0]
else:
    noise_count = 0

noise_pct = (noise_count / total_docs) * 100

print(f"Reduced Model - Total Topics (excluding noise): {n_topics}")
print(f"Noise Documents (Topic -1): {noise_count} ({noise_pct:.2f}%)")


Reduced Model - Total Topics (excluding noise): 19
Noise Documents (Topic -1): 3346 (33.60%)


In [38]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

# Tokenized documents
tokenized_docs = [doc.split() for doc in docs]

# Create dictionary
dictionary = Dictionary(tokenized_docs)

# Get top words from reduced model
reduced_topics = reduced_model.get_topics()
reduced_top_words = [
    [word for word, _ in reduced_topics[i][:10]]
    for i in reduced_topics if i != -1
]

# Coherence calculation
reduced_coh_model = CoherenceModel(
    topics=reduced_top_words,
    texts=tokenized_docs,
    dictionary=dictionary,
    coherence='c_v'
)

reduced_coherence_score = reduced_coh_model.get_coherence()
print(f"Reduced Model Coherence (c_v): {reduced_coherence_score:.4f}")

Reduced Model Coherence (c_v): 0.5238


In [39]:
# Get topics from reduced model
reduced_topics = reduced_model.get_topics()
top_n = 10  # number of top words per topic to consider

# Collect all top words
all_words = []
for topic in reduced_topics.values():
    if topic:
        all_words.extend([word for word, _ in topic[:top_n]])

# Calculate diversity
unique_words = set(all_words)
diversity_score = len(unique_words) / len(all_words)

print(f"Reduced Model Topic Diversity: {diversity_score:.4f}")

Reduced Model Topic Diversity: 0.7050


In [20]:
reduced_model.save("bertopic_model_trial_8_reduced", save_embedding_model=True)
print("✅ Reduced model saved as 'bertopic_model_trial_8_reduced'")

2025-07-03 21:20:42,829 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


✅ Reduced model saved as 'bertopic_model_trial_8_reduced'
